In [5]:
import os, sys
import pandas as pd
import numpy as np

sys.path.append('../')
from knobs import Knob
from utils import rocksdb_knobs_make_dict
from steps import train_fitness_function
from torch.utils.data import DataLoader
from network import RocksDBDataset, SingleNet
from train import train, valid

In [7]:
class OPT():
    def __init__(self):
        self.target = 0
        self.batch_size = 32
        self.epochs = 20
        self.lr = 0.0001
        
opt = OPT()

In [11]:
KNOB_PATH = f'../data/mysql/configs/{opt.target}'
EXTERNAL_PATH = '../data/mysql/external'
INTERNAL_PATH = '../data/mysql/internal'
WK_NUM = 1

In [151]:
def mysql_knob_dataframe(knobs_path):
    config_len = len(os.listdir(knobs_path))
    cnf_parser = configparser.ConfigParser()
    pd_mysql = pd.DataFrame()
    for idx in range(config_len):
        cnf_parser.read(os.path.join(knobs_path, f'my_{idx}.cnf'))
        conf_dict = cnf_parser._sections['mysqld']
        tmp = pd.DataFrame(data=[conf_dict.values()], columns=conf_dict.keys())
        pd_mysql = pd.concat([pd_mysql, tmp])
        
    pd_mysql = pd_mysql.reset_index(drop=True)
    pd_mysql = pd_mysql.drop(columns=['log-error', 'bind-address'])
    return pd_mysql

def mysql_metrics_dataframe(wk, internal_path, external_path):
    internal = pd.read_csv(os.path.join(internal_path, f'internal_results_{wk}.csv'), index_col=0)
    ## Drop oolumns contained unique data
    unique_data_column = []
    for col in internal.columns:
        if len(pd.value_counts(internal[col])) == 1:
            unique_data_column.append(col)
    internal = internal.drop(columns=unique_data_column)
    
    external = pd.read_csv(os.path.join(external_path, f'external_results_{wk}.csv'), index_col=0)
    latency_columns = []
    for col in external.columns:
        if col.find("latency") == 0 and col != 'latency_max' and col != 'latency_CLEANUP':
            latency_columns.append(col)
    external_ = external[['tps']].copy()
    external_['latency'] = external[latency_columns].max(axis=1)
    return internal, external_

In [60]:
raw_knobs = mysql_dataframe(KNOB_PATH)

In [152]:
wk = 0
internal, external = mysql_metrics_dataframe(wk, INTERNAL_PATH, EXTERNAL_PATH)

In [154]:
external

,tps,latency
0,87624.7,305.0
1,86652.8,339.0
2,86010.4,431.0
3,79557.0,362.0
4,83238.3,398.0
...,...,...
995,84583.0,328.0
996,92383.8,309.0
997,88124.3,379.0
998,79968.6,363.0


In [62]:
wk = 0
internal = pd.read_csv(os.path.join(INTERNAL_PATH, f'internal_results_{wk}.csv'), index_col=0)

unique_data_column = []
for col in internal.columns:
    if len(pd.value_counts(internal[col])) == 1:
        unique_data_column.append(col)
internal = internal.drop(columns=unique_data_column)

In [146]:
external = pd.read_csv(os.path.join(EXTERNAL_PATH, f'external_results_{wk}.csv'), index_col=0)

In [147]:
latency_columns = []
for col in external.columns:
    if col.find("latency") == 0 and col != 'latency_max' and col != 'latency_CLEANUP':
        latency_columns.append(col)

In [148]:
external_ = external[['tps']].copy()

In [149]:
external_['latency'] = external[latency_columns].max(axis=1)

In [150]:
external_

,tps,latency
0,87624.7,305.0
1,86652.8,339.0
2,86010.4,431.0
3,79557.0,362.0
4,83238.3,398.0
...,...,...
995,84583.0,328.0
996,92383.8,309.0
997,88124.3,379.0
998,79968.6,363.0


In [102]:
latency_columns

['latency_READ', 'latency_UPDATE']

In [88]:
external.columns.to_list().find('latency')

AttributeError: 'list' object has no attribute 'find'

In [84]:
external.columns.find('latency')

AttributeError: 'Index' object has no attribute 'find'

In [74]:
pd.value_counts(internal['ibuf_merges_insert'])

0    1000
Name: ibuf_merges_insert, dtype: int64

In [63]:
internal

,os_data_reads,buffer_pages_read,dml_reads,dml_updates,buffer_pool_bytes_data,innodb_dblwr_pages_written,buffer_data_reads,buffer_pool_pages_free,buffer_pool_read_ahead_evicted,adaptive_hash_searches_btree,...,os_log_pending_writes,ibuf_merges_delete_mark,buffer_pool_pages_data,buffer_pool_reads,buffer_pool_write_requests,os_log_fsyncs,buffer_pages_created,innodb_rwlock_s_spin_waits,os_data_writes,innodb_activity_count
0,5801394,5802332,10000000,0,2545451008,160,95025090560,3,0,8869669,...,0,0,155362,5800081,39063,334,0,1588762,640,0
1,5267339,5268146,10000000,0,3515121664,125,86285078528,0,0,8577293,...,0,0,214546,5266145,39063,356,0,1228184,606,0
2,4119631,4122830,10000000,0,7957610496,123,67430846464,0,0,9999941,...,0,0,485694,4119911,39063,336,0,4728,577,0
3,6429437,6430420,10000000,0,1932640256,190,105302724608,0,0,9137076,...,0,0,117959,6427945,39063,438,0,1730471,818,0
4,4920091,4921076,10000000,0,4584095744,168,80592719872,0,0,8383075,...,0,0,279791,4918587,39063,399,0,1122916,733,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5489151,5489902,10000000,0,3017293824,165,89907990528,0,8447,8690383,...,0,0,184161,5487818,39063,303,0,1399734,633,0
996,5234987,5237867,10000000,0,3686055936,116,85579829248,0,0,9999362,...,0,0,224979,5233819,39063,269,0,2665,501,0
997,4674370,4677289,10000000,0,6210977792,163,76467736576,0,115562,10000364,...,0,0,379088,4674012,39063,284,0,2806,610,0
998,5822417,5823206,10000000,0,2425929728,138,95364337664,0,2156,8894439,...,0,0,148067,5821233,39063,303,0,1534786,560,0


In [5]:
internal_dict = {}
external_dict = {}

pruned_im = pd.read_csv(os.path.join(INTERNAL_PATH, 'internal_ensemble_pruned_tmp.csv'), index_col=0)
for wk in range(WK_NUM):
    im = pd.read_csv(os.path.join(INTERNAL_PATH, f'internal_results_{wk}.csv'), index_col=0)
    internal_dict[wk] = im[pruned_im.columns]
if opt.target > 15:
    im = pd.read_csv(f'data/target_workload/{opt.target}/internal_results_11.csv', index_col=0)
    internal_dict[wk+1] = im[pruned_im.columns]

for wk in range(WK_NUM):
    ex = pd.read_csv(os.path.join(EXTERNAL_PATH, f'external_results_{wk}.csv'), index_col=0)
    external_dict[wk] = ex
if opt.target > 15:
    ex = pd.read_csv(f'data/target_workload/{opt.target}/external_results_11.csv', index_col=0)
    external_dict[wk+1] = ex

In [6]:
knobs = Knob(raw_knobs, internal_dict, external_dict, opt.target)

In [7]:
knobs.split_data()
knobs.scale_data()

In [8]:
dataset_tr = RocksDBDataset(knobs.norm_X_tr, knobs.norm_em_tr)
dataset_te = RocksDBDataset(knobs.norm_X_te, knobs.norm_em_te)

loader_tr = DataLoader(dataset = dataset_tr, batch_size = opt.batch_size, shuffle=True)
loader_te = DataLoader(dataset = dataset_te, batch_size = opt.batch_size, shuffle=False)

model = SingleNet(input_dim=knobs.norm_X_tr.shape[1], hidden_dim=16, output_dim=knobs.norm_em_tr.shape[-1]).cuda()

In [10]:
best_loss = 100
# name = get_filename('model_save', 'model', '.pt')
for epoch in range(opt.epochs):
    loss_tr = train(model, loader_tr, opt.lr)
    loss_te, outputs = valid(model, loader_te)

    print(f"[{epoch:02d}/{opt.epochs}] loss_tr: {loss_tr:.8f}\tloss_te:{loss_te:.8f}")

[00/20] loss_tr: 0.04003805	loss_te:0.03580615
[01/20] loss_tr: 0.03285220	loss_te:0.02912505
[02/20] loss_tr: 0.02611693	loss_te:0.02275005
[03/20] loss_tr: 0.02022883	loss_te:0.01747382
[04/20] loss_tr: 0.01521571	loss_te:0.01291721
[05/20] loss_tr: 0.01110809	loss_te:0.00943629
[06/20] loss_tr: 0.00824655	loss_te:0.00725319
[07/20] loss_tr: 0.00680578	loss_te:0.00643578
[08/20] loss_tr: 0.00633372	loss_te:0.00619593
[09/20] loss_tr: 0.00614985	loss_te:0.00605251
[10/20] loss_tr: 0.00604046	loss_te:0.00596460
[11/20] loss_tr: 0.00596313	loss_te:0.00590390
[12/20] loss_tr: 0.00591374	loss_te:0.00584863
[13/20] loss_tr: 0.00587182	loss_te:0.00580835
[14/20] loss_tr: 0.00584023	loss_te:0.00577448
[15/20] loss_tr: 0.00581302	loss_te:0.00576012
[16/20] loss_tr: 0.00578510	loss_te:0.00570342
[17/20] loss_tr: 0.00574415	loss_te:0.00564570
[18/20] loss_tr: 0.00567817	loss_te:0.00556786
[19/20] loss_tr: 0.00559141	loss_te:0.00549298


In [49]:
def single_score_function(df, pr):
    df = np.repeat(df, pr.shape[0], axis=0)
    score = (df[0] - pr[0]) + (pr[1] - df[1]) + (df[2] - pr[2]) + (df[3] - pr[3])
    print(score)
    return np.round(score, 6)

In [53]:
df = np.repeat(knobs.default_trg_em, outputs.shape[0], axis=0)
pr = outputs.detach().cpu().numpy()

In [61]:
(df[:,0] - pr[:,0] + pr[:,1] - df[:,1])

array([-0.33032445, -0.4273648 , -0.35677413, ..., -0.21283447,
       -0.36919947, -0.12358297])

In [63]:
0.00362396 - 0.02582621 + 0.42511803 - 0.73324022

-0.33032444

In [62]:
df, pr

(array([[0.00362396, 0.73324022, 0.23343849, 0.55314511],
        [0.00362396, 0.73324022, 0.23343849, 0.55314511],
        [0.00362396, 0.73324022, 0.23343849, 0.55314511],
        ...,
        [0.00362396, 0.73324022, 0.23343849, 0.55314511],
        [0.00362396, 0.73324022, 0.23343849, 0.55314511],
        [0.00362396, 0.73324022, 0.23343849, 0.55314511]]),
 array([[ 0.02582621,  0.42511803,  0.21571767,  0.32778028],
        [ 0.03594913,  0.3382006 ,  0.11593885,  0.5312192 ],
        [ 0.02982348,  0.40266562,  0.34310928,  0.8430038 ],
        ...,
        [ 0.02072407,  0.53750587,  0.19755192,  0.86267316],
        [ 0.0325234 ,  0.3929402 ,  0.20183249,  0.4707964 ],
        [-0.00550541,  0.6005279 ,  0.20492807,  1.0165479 ]],
       dtype=float32))

In [50]:
single_score_function(knobs.default_trg_em, outputs.detach().cpu().numpy())

[-0.02957605  0.43930064 -0.25100482 -0.53670648]


array([-0.029576,  0.439301, -0.251005, -0.536706])

In [42]:
knobs.default_trg_em

array([[0.00362396, 0.73324022, 0.23343849, 0.55314511]])

In [44]:
np.repeat(knobs.default_trg_em, outputs.shape[0], axis=0)

array([[0.00362396, 0.73324022, 0.23343849, 0.55314511],
       [0.00362396, 0.73324022, 0.23343849, 0.55314511],
       [0.00362396, 0.73324022, 0.23343849, 0.55314511],
       ...,
       [0.00362396, 0.73324022, 0.23343849, 0.55314511],
       [0.00362396, 0.73324022, 0.23343849, 0.55314511],
       [0.00362396, 0.73324022, 0.23343849, 0.55314511]])

In [35]:
outputs.detach().cpu().numpy()

array([[ 0.02582621,  0.42511803,  0.21571767,  0.32778028],
       [ 0.03594913,  0.3382006 ,  0.11593885,  0.5312192 ],
       [ 0.02982348,  0.40266562,  0.34310928,  0.8430038 ],
       ...,
       [ 0.02072407,  0.53750587,  0.19755192,  0.86267316],
       [ 0.0325234 ,  0.3929402 ,  0.20183249,  0.4707964 ],
       [-0.00550541,  0.6005279 ,  0.20492807,  1.0165479 ]],
      dtype=float32)

In [23]:
single_score_function(knobs.default_trg_em, outputs.cpu().numpy())

-0.087239

In [15]:
knobs.default_trg_em

array([[0.00362396, 0.73324022, 0.23343849, 0.55314511]])

In [22]:
outputs[0].cpu().numpy()

array([0.02582621, 0.42511803, 0.21571767, 0.32778028], dtype=float32)

In [31]:
np.round(outputs.cpu().detach().numpy()[0], 6)

array([0.025826, 0.425118, 0.215718, 0.32778 ], dtype=float32)

tensor([[ 0.0258,  0.4251,  0.2157,  0.3278],
        [ 0.0359,  0.3382,  0.1159,  0.5312],
        [ 0.0298,  0.4027,  0.3431,  0.8430],
        ...,
        [ 0.0207,  0.5375,  0.1976,  0.8627],
        [ 0.0325,  0.3929,  0.2018,  0.4708],
        [-0.0055,  0.6005,  0.2049,  1.0165]], device='cuda:0')

In [12]:
outputs.shape, outputs

(torch.Size([4000, 4]),
 tensor([[ 0.0258,  0.4251,  0.2157,  0.3278],
         [ 0.0359,  0.3382,  0.1159,  0.5312],
         [ 0.0298,  0.4027,  0.3431,  0.8430],
         ...,
         [ 0.0207,  0.5375,  0.1976,  0.8627],
         [ 0.0325,  0.3929,  0.2018,  0.4708],
         [-0.0055,  0.6005,  0.2049,  1.0165]], device='cuda:0'))